# Step 1: Create an amplitude model

[tensorwaves](tensorwaves) requires you to first formulate an amplitude model that you want to fit to your data set. The [expertsystem](expertsystem) helps you to construct such a model.

This notebook briefly illustrates how to create such an amplitude model with the [expertsystem](expertsystem) and how to write it to a recipe file that can be understood by [tensorwaves](tensorwaves). We won't go into details here, because the [expertsystem](expertsystem) already provides {doc}`its own guides <expertsystem:usage>`.

We first define the boundary conditions of our physics problem, that is, initial state, final state, formalism type, and the intermediate states that we're interested in. All this is handled through the [StateTransitionManager](expertsystem.reaction.StateTransitionManager) (STM). In this example, we chose to use the helicity formalism, but you can also use `formalism_type="canonical-helicity"`. As you can see, we analyze the decay $J/\psi \to \pi^0\pi^0\gamma$ here.

In [ ]:
from expertsystem.reaction import InteractionTypes, StateTransitionManager

stm = StateTransitionManager(
    initial_state=[("J/psi(1S)", [-1, +1])],
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)"],
    formalism_type="helicity",
)
stm.set_allowed_interaction_types([InteractionTypes.Strong])

```{admonition} Simplified model: $J/\psi \to f_0\gamma$
---
class: dropdown
---
As {doc}`3_perform_fit` serves to illustrate usage only, we make the amplitude model here a bit simpler by not allowing $\omega$ resonances (which are narrow and therefore hard to fit). For this reason, we can also limit the {class}`~expertsystem.reaction.solving.InteractionTypes` to {attr}`~expertsystem.reaction.solving.InteractionTypes.Strong`.
```

Next, we let the [STM](expertsystem.reaction.StateTransitionManager) generate topologies (represented by graphs) for the transition from initial to final state. We're not really interested in warnings (see {doc}`the ExpertSystem <expertsystem:usage>` for more details), so we suppress them using [logging](logging).

In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.ERROR)
graph_interaction_settings_groups = stm.prepare_graphs()

Now we're ready to let the STM compute the allowed intermediate states. In this example, we just use the standard interaction type settings that [StateTransitionManager](expertsystem.reaction.StateTransitionManager) generated.

In [ ]:
result = stm.find_solutions(graph_interaction_settings_groups)

Have a look at {doc}`expertsystem:usage/visualization` if you want to visualize the decay topologies that the [expertsystem](expertsystem) has found.

Finally, we're ready to create an [AmplitudeModel](expertsystem.amplitude.model.AmplitudeModel). This can be done with the [generate_amplitude_model](expertsystem.amplitude.generate_amplitude_model) method.

In [ ]:
from expertsystem.amplitude import generate_amplitude_model

model = generate_amplitude_model(result)

Finally, we can write the [AmplitudeModel](expertsystem.amplitude.model.AmplitudeModel), using the [write](expertsystem.io.write) function:

In [ ]:
from expertsystem import io

io.write(model, "amplitude_model_helicity.yml")

Cool, that's it! We now have a recipe for an amplitude model with which to [generate data](./2_generate_data) and [perform a fit](./3_perform_fit)! In the [next steps](./2_generate_data), we will use use this [AmplitudeModel](expertsystem.amplitude.model.AmplitudeModel) as a fit model template for [tensorwaves](tensorwaves).